# Inserting ISS Data To MySQL with SQLAlchemy

## Collecting data from the API

In [1]:
import requests
url =  "http://api.open-notify.org/iss-now.json"
response = requests.get(url)
json_response = response.json()
json_response

{'iss_position': {'longitude': '86.3481', 'latitude': '-3.5258'},
 'message': 'success',
 'timestamp': 1637869196}

Extract only the pieces of info we need and store them in a dictionary

In [2]:
data_to_insert = {"latitude":json_response["iss_position"]["latitude"],
                  "longitude":json_response["iss_position"]["longitude"],
                  "iss_timestamp":json_response["timestamp"]}
data_to_insert

{'latitude': '-3.5258', 'longitude': '86.3481', 'iss_timestamp': 1637869196}

Timestamp from UNIX to proper datetime format

In [3]:
from datetime import datetime
data_to_insert["iss_timestamp"] = datetime.utcfromtimestamp(data_to_insert["iss_timestamp"]).strftime('%Y-%m-%d %H:%M:%S')
data_to_insert

{'latitude': '-3.5258',
 'longitude': '86.3481',
 'iss_timestamp': '2021-11-25 19:39:56'}

Create a pandas Dataframe with the same columns as the SQL table we want to have in our database

In [4]:
import pandas as pd

iss_df = pd.DataFrame({
    "latitude":[],
    "longitude":[],
    "iss_timestamp":[]
})
iss_df

,latitude,longitude,iss_timestamp


Append data to DataFrame

In [5]:
iss_df = iss_df.append(data_to_insert, ignore_index=True)
iss_df

,latitude,longitude,iss_timestamp
0,-3.5258,86.3481,2021-11-25 19:39:56


# SQLAlchemy

In [6]:
!pip install PyMySql

In [14]:
import sqlalchemy # install if needed
from api_keys import local_mysql_user
from api_keys import local_mysql_password

print(local_mysql_password)

ImportError: cannot import name 'local_mysql_user' from 'api_keys' (/home/donal/WBS_coding_school_data_science_bootcamp/wk-6_data_pipelines/api_keys.py)

Specify MySQL connection. You need to previously create the schema (also called database in MySQL) `iss_workshop` on your local instance of MySQL. 

You can do that with MySQLWorkbench by connecting to your local instance and typing `CREATE DATABASE iss_workshop;` in a new query tab.

In [3]:
database="iss_workshop"
host="127.0.0.1"
user=local_mysql_user
password=local_mysql_password
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

Use pandas method `to_sql` with the argument `if_exists=append` to create the table (only the first time we run it) and insert the new rows into it.

In [10]:
iss_df.to_sql('iss_position', 
              if_exists='append', 
              con=con, 
              index=False)

Check on MySQLWorkbench that a new table `iss_position` exists within the `iss_workshop` database, and that a new row has been inserted on it. If you run the whole notebook again, another row should appear there.